In [97]:
from datetime import datetime,timedelta,date
import random,os,calendar
import numpy as np
import pandas as pd
import logging

#기간내 랜덤일자 추출
def random_date(start, end):
    return start + timedelta(
          seconds=random.randint(0, int((end - start).total_seconds())),  )

##몇년,몇월,몇일 전/후
def date_add(origin_dt,**kwargs):
    if 'days' in kwargs:
        add_val= int(kwargs.get('days'))
        return origin_dt + timedelta(days=add_val)
    elif 'years' in kwargs:
        add_val= int(kwargs.get('years'))* 12
    elif 'months' in kwargs:
        add_val= int(kwargs.get('months'))
    else: return ''
    
    new_year=origin_dt.year
    new_month= origin_dt.month + add_val 
    if new_month < 0:
          yrs= -(abs(add_val) // 12)
    else: yrs= new_month // 12
    new_year += yrs
    new_month = new_month - yrs * 12
    if new_month == 0:
        new_year += -1
        new_month = 12

    last_day_of_month=calendar.monthrange(new_year,new_month)[1]
    new_day=min(origin_dt.day, last_day_of_month)
    return date(new_year,new_month,new_day)
         
#check digit
def make_chk(rsNo):
    # 리스트끼리 곱 (np이용, list연산은 더하기만 가능)
    # string각 요소를 int형로 변환 map(int, list(rsNo)) 
    i1=np.array(list(map(int, list(rsNo))))
    i2=np.array([2,3,4,5,6,7,8,9,2,3,4,5])
    # chk= 11에서 각 리스트 곱의 합을 11로 나눈 나머지를 뺀 값(나머지>=10 이면 10을빼줌)
    chk= 11 - np.sum(i1*i2) % 11
    chk=chk if chk < 10 else chk-10
    return chk

#주민번호 생성
def make_rsno(dt):
    strDt=str(dt.year)[2:]+str(dt.month).zfill(2)+str(dt.day).zfill(2)
    #성별 /지역번호
    strSx=str(random.randrange(1,3) if dt.year<2000 else random.randrange(3,5))
    strReg=str(random.randrange(10000, 99999))
    rsNo=strDt+strSx+strReg
    return rsNo+str(make_chk(rsNo))

In [ ]:
for i in range(10):
    rsno=make_rsno(random_date(date(1960,1,1),date(2001,12,31)))
    print(rsno)

In [ ]:
#차량번호
for i in range(10):
    gana=['가','나','다','라','마','보','거','구','조']
    print(str(random.randrange(11,70)).zfill(2),random.choice(gana),str(random.randrange(1,9999)).zfill(4))

In [271]:
def make_crno():
    gana=['가','나','다','라','마','보','거','구','조']
    return str(random.randrange(11,70)).zfill(2)+random.choice(gana)+str(random.randrange(1,9999)).zfill(4)

def make_dvpns_plyno():
    strReg=str(random.randrange(0,999999999999))
    return strReg.zfill(12)

def make_val(cstr,clen):
    choice=cstr.split(',')
    return random.choice(choice).rjust(clen,' ')

def make_date(val):
    strdt1,strdt2 =val.split('~')
    dt= random_date(datetime.strptime(strdt1,'%Y%m%d'),datetime.strptime(strdt2,'%Y%m%d'))
    return datetime.strftime(dt,'%Y%m%d')

def make_date2(strDt,**kwargs):
    dt=datetime.strptime(strDt,'%Y%m%d')
    for key,value in kwargs.items():
        if key=='year':
            dt=date(dt.year+int(value),dt.month,dt.day)
        elif key=='month':    
            dt=add_month(dt,int(value))
    return datetime.strftime(dt,'%Y%m%d')   

In [101]:
# make_val('11,21,22',2)
# make_date()
# datetime.strftime(date(2021, 1, 30),'%Y%m%d')
# datetime.strptime('20200130','%Y%m%d')
# make_clstr('20200130',year=1)
dt+timedelta(days=3)

datetime.datetime(2020, 2, 2, 0, 0)

In [273]:
#excel의 layout과 컬럼명 기본값(무작위선택 리스트 포함)읽어 tsv파일을 생성
#sample: d:/kpayins/temp/갱신율.xlsx 참조
layout=pd.read_excel('d:/kpayins/temp/갱신율.xlsx',sheet_name='layout',dtype={'col_val': str},header=0)     
createCnt=5
try:
    df=pd.DataFrame()
    for i in range(0,createCnt):
        dfT=pd.DataFrame()
        for seq, row in layout.iterrows():
    #         print('seq=',seq)
            row=row.fillna('')
            ran_yn= str(row['random_yn']).lower()
            col_len=int(row['col_len'])
            col_id= row['col_id']
            col_val= row['col_val']
            if ran_yn=='':
                val= col_val.ljust(col_len,' ')
            elif ran_yn=='rsno':
                val=make_rsno(random_date(date(1960,1,1),date(2001,12,31)))
            elif ran_yn=='crno':
                val=make_crno()
            elif ran_yn=='plyno':
                val=make_dvpns_plyno()
            elif ran_yn=='copy':   # 타 컬럼으로 부터 값 복사
                from_col=col_val
                val=dfT[from_col].values[0]
            elif ran_yn=='random_date':
                val=make_date(col_val)   
            elif ran_yn.find('+') > 0:  # 특정컬럼으로 부터 계산값 만드는것 : '컬럼+계산단위'로 입력
                from_col,from_unit =ran_yn.split('+')
                if from_unit=='year':
                    val=make_date2(dfT[from_col].values[0],year=col_val)
                elif from_unit=='month':
                    val=make_date2(dfT[from_col].values[0],month=col_val)    
            elif ran_yn=='val':
                val=make_val(col_val,col_len)
            elif ran_yn=='nm':
                val='테스형'+str(i)

            val=val.ljust(col_len,' ')
            dfT[col_id]=[val]
        df=pd.concat([df,dfT])
        
    wrt_file='d:/kpayins/temp/ss.dat'
    df.to_csv(wrt_file, index=False,sep='\t',encoding='utf-8') 
    print("job completed~~")
    
except  Exception as e: 
        import time
        print(f'error occured..{str(e)}')
        logging.info('%s ← error occured at..%s' %(str(e),time.strftime("%Y%m%d %H:%M:%S",time.localtime())))
        pass

job completed~~


In [ ]:
dt= random_date(date(2021,1,1),date(2021,1,31))
dt= date(2020,2,26)
dt=date(dt.year+1,dt.month,dt.day)
datetime.strftime(dt,'%Y%m%d')

In [291]:
# calendar.monthrange(2020,2)
dt=date(2020,2,29)
dt.replace(day=1)

datetime.date(2020, 2, 1)

In [267]:
##몇 개월 전/후
def add_month2(origin_dt, add_val):
    new_year=origin_dt.year
    new_month= origin_dt.month + add_val 
    if new_month < 0:
        yrs= -(abs(add_val) // 12)
        new_year += yrs
        new_month = new_month - yrs * 12
    elif new_month > 12:
        yrs= new_month // 12
        new_year += yrs
        new_month = new_month - yrs * 12
        
    if new_month == 0:
        new_year += -1
        new_month = 12

    last_day_of_month=calendar.monthrange(new_year,new_month)[1]
    new_day=min(origin_dt.day, last_day_of_month)
    return date(new_year,new_month,new_day)
        

In [98]:
print(date_add(date(2020,12,31),months=12))
print(date_add(date(2020,12,31),months=-30))
print(date_add(date(2022,3,30),years=-1))
print(date_add(date(2020,12,31),days=2))

2021-12-31
2018-06-30
2021-03-30
2021-01-02


In [29]:
def ss(origin_dt, param):
    a,b=param.split('=')
    print(a)

ss('a','year=1')

year


In [85]:
def ss(strDt,**kwargs):
    if 'year' in kwargs:
        print(kwargs.get('year'))
    
ss('a',year=1)

1


In [75]:
**kwargs?